In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

df = pd.read_csv('../data/supplements_20250311.csv')

In [4]:
# 키워드 목록 (사용자가 찾을 가능성이 높은 주요 기능성)
keywords = ['비타민']

# 키워드별로 제품을 필터링 후 CSV로 저장
for keyword in keywords:
    filtered_df = df[df['기능성'].str.contains(keyword, case=False, na=False)]
    
    if not filtered_df.empty:
        file_name = f'preprocessed_{keyword.replace(" ", "_")}.csv'
        filtered_df.to_csv(file_name, index=False, encoding='utf-8-sig')
        print(f"📂 '{keyword}' 관련 제품을 '{file_name}'로 저장 완료!")

📂 '비타민' 관련 제품을 'preprocessed_비타민.csv'로 저장 완료!


In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# CSV 파일 로드
df = pd.read_csv('preprocessed_비타민.csv')

# 결측값 제거
df = df.dropna(subset=['기능성'])

# TF-IDF 벡터화
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['기능성'])

# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# **📌 키워드를 TF-IDF로 검색하는 방식으로 변경**
def recommend(keyword, limit=5):
    """키워드 기반 제품 추천"""
    # 🔹 키워드를 TF-IDF로 변환
    keyword_vec = tfidf.transform([keyword])

    # 🔹 키워드와 모든 제품 간 코사인 유사도 계산
    sim_scores = cosine_similarity(keyword_vec, tfidf_matrix)[0]

    # 🔹 유사도가 높은 상위 limit개 제품 가져오기
    top_indices = sim_scores.argsort()[-limit:][::-1]

    return df[['제품명']].iloc[top_indices]

# 예시 테스트: '비타민'과 관련된 추천 제품
recommend_results = recommend('비타민K', limit=9)
print(recommend_results)




                       제품명
2436               철분 프리미엄
2229                   잇튼튼
3281              활성엽산 영양제
2223                    잇센
3173  L-아르기닌 에너지 분말(전량수출용)
2228                   잇타민
2557      코엔자임큐텐 맥스 100 mg
2548               코엔자임Q10
98               덴티모어 임플타민


In [36]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# CSV 파일 로드
df = pd.read_csv('preprocessed_비타민.csv')

# 결측값 제거
df = df.dropna(subset=['기능성', '제품명'])

# 기능성과 제품명을 결합하여 하나의 텍스트 컬럼 생성
df['기능성_제품명'] = df['기능성'] + " " + df['제품명']

# TF-IDF 벡터화
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['기능성_제품명'])

# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# **📌 키워드를 TF-IDF로 검색하는 방식으로 변경**
def recommend(keyword, limit=5):
    """키워드 기반 제품 추천"""
    # 🔹 키워드를 TF-IDF로 변환
    keyword_vec = tfidf.transform([keyword])

    # 🔹 키워드와 모든 제품 간 코사인 유사도 계산
    sim_scores = cosine_similarity(keyword_vec, tfidf_matrix)[0]

    # 🔹 유사도가 높은 상위 limit개 제품 가져오기
    top_indices = sim_scores.argsort()[-limit:][::-1]

    return df[['제품명']].iloc[top_indices]

# 예시 테스트: '비타민'과 관련된 추천 제품
recommend_results = recommend('비타민D', limit=9)
print(recommend_results)


                           제품명
303            리포좀 비타민C & 비타민D
3397  고려은단 비타민C 1000 구미 + 비타민D
306        리포좀 비타민C + 리포좀 비타민D
706          면역기능에 필요한 아연 비타민D
2746                 튼튼아연+비타민D
3401     고려은단 비타민C1000 이지+비타민D
1495       써니 비타민D 플러스C (복숭아맛)
3402     고려은단 비타민C1000 이지+비타민D
22             더블 업 철분 엽산 비타민D
